### IMPORTS:

In [25]:
import pandas as pd
import numpy as np
import os

## Juntando os datasets

### CONCAT:

In [26]:
path = 'CSVs/'
files = os.listdir(path=path)

df = pd.concat([pd.read_csv(f'{path}/{file}') for file in files])
df.to_csv(f'{path}/one4all.csv')

---

### Reset pré cleanup (Caso necessário)

In [ ]:
df = pd.read_csv('CSVs/one4all.csv')

---
## DATA PREPARATION

### DATA CLEANUP:

#### Arrumando os nomes começando com espaços

In [27]:
df.rename(columns= lambda s: s.strip(), inplace=True)

#### Removendo colunas com NaN

In [28]:
print(len(df))
df.dropna(inplace=True)
print(len(df))

2830743
2829385


#### Removendo as Features não significativas 

In [30]:
columns = [
    'Bwd PSH Flags',
    'Bwd URG Flags', 
    'Fwd Avg Bytes/Bulk',
    'Fwd Avg Packets/Bulk',
    'Fwd Avg Bulk Rate',
    'Bwd Avg Bytes/Bulk', 
    'Bwd Avg Packets/Bulk',
    'Bwd Avg Bulk Rate'
]

print(len(df.columns))
df.drop(columns=columns, inplace=True)
print(len(df.columns))

79
71
